In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sh
ls /dbfs/mnt/demo-datasets/DE-Pro-new/bookstore/kafka-raw/

In [0]:
def process_bronze():
  
    schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"

    query = (spark.readStream
                        .format("cloudFiles")
                        .option("cloudFiles.format", "json")
                        .schema(schema)
                        .load(f"{dataset_bookstore}/kafka-raw")
                        .withColumn("timestamp", (F.col("timestamp")/1000).cast("timestamp"))  
                        .withColumn("year_month", F.date_format("timestamp", "yyyy-MM"))
                  .writeStream
                      .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/bronze")
                      .option("mergeSchema", True)
                      .partitionBy("topic", "year_month")
                      .trigger(availableNow=True)
                      .table("bronze"))
    
    query.awaitTermination()

In [0]:
%sql
-- drop table new_bronz;
show tables;

In [0]:
def process_bronze():
    schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"
    # schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"

    query =   (spark.readStream
                    .format("cloudFiles")
                    .option("cloudFiles.format", "json")
                    .schema(schema)
                    .load("dbfs:/mnt/demo-datasets/DE-Pro-new/bookstore/kafka-raw")
                    .withColumn("timestamp", (col("timestamp")/1000).cast("timestamp"))
                    .withColumn("year_month", date_format("timestamp", "yyyy-MM"))
                .writeStream
                    .option("checkpointLocation", "dbfs:/mnt/checkpoints/bronze")
                    .option("mergeSchema", True)
                    .partitionBy("topic", "year_month")
                    .trigger(availableNow=True)
                    .table("bronze_table")
        )

    query.awaitTermination()    


In [0]:
# %sh
# rm -rf /dbfs/mnt/checkpoints/bronze

In [0]:
process_bronze()

In [0]:
batch_df = spark.table("bronze_table")
display(batch_df)

In [0]:
%sql
select * from bronze_table;

In [0]:
json_schema = "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>"

query = (spark.readStream.table("bronze_table")
        .filter("topic = 'orders'")
        .select(from_json(col("value").cast("string"), json_schema).alias("v"))
        .select("v.*")
        .filter("quantity > 0")
     .writeStream
        .option("checkpointLocation", "dbfs:/mnt/checkpoints/silver")
        .trigger(availableNow=True)
        .table("silver_table"))

query.awaitTermination()

In [0]:
%sql
select * from silver_table;